In [11]:
import os
import time
import re
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import pandas as pd
import json
import uuid
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer,BertTokenizerFast, BertForTokenClassification
import datasets
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import ServerlessSpec
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as LangChainPinecone
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings


from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="project3"


In [59]:


def clean_text(text):
    """
    Clean the extracted text by removing unnecessary characters and formatting.
    
    Args:
    - text (str): The extracted text.
    
    Returns:
    - str: The cleaned text.
    """
    # Remove multiple spaces and replace with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove line breaks and tabs
    text = text.replace('\n', ' ').replace('\t', ' ')
    # Remove non-alphanumeric characters and basic punctuation
    text = re.sub(r'[^a-zA-Z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ.,;!?()\-\'" ]', '', text)
    return text

def find_pdfs_and_jsons(directory):
    """
    Recursively find all PDFs and their associated JSON files in a directory.
    
    Args:
    - directory (str): The base directory to search.
    
    Returns:
    - list: A list of tuples, each containing the paths to a PDF file and its associated JSON files.
    """
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.lower().endswith('.pdf'):
                pdf_path = os.path.join(root, filename)
                llm_output_path = os.path.join(root, "expected-llm-output.json")
                services_output_path = os.path.join(root, "expected-services-output.json")
                if os.path.exists(llm_output_path) and os.path.exists(services_output_path):
                    files.append((pdf_path, llm_output_path, services_output_path))
    return files

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file using PyMuPDF (fitz).
    Use OCR for scanned pages.
    
    Args:
    - pdf_path (str): Path to the PDF file.
    
    Returns:
    - str: Extracted text.
    """
    text = ""
    document = fitz.open(pdf_path)
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
        if not text.strip():  # If no text is extracted, use OCR
            pix = page.get_pixmap()
            img = Image.open(io.BytesIO(pix.tobytes()))
            text += pytesseract.image_to_string(img)
    cleaned_text = clean_text(text)
    return cleaned_text



In [60]:
# Path to the base directory
base_directory = "data/pdfs"

pdf_data = []
file_paths = find_pdfs_and_jsons(base_directory)

# Create a list to hold the data
data = []

for pdf_file, llm_output_file, services_output_file in file_paths:
    text = extract_text_from_pdf(pdf_file)
    
    with open(llm_output_file, 'r', encoding='utf-8') as f:
        llm_output = json.load(f)
        
    with open(services_output_file, 'r', encoding='utf-8') as f:
        services_output = json.load(f)
    
    for deliberacao in llm_output.get("deliberacoes", []):
        for service_info in services_output:
            if service_info["deliberacao"].lower() == deliberacao.lower():
                data.append({
                    "id": str(uuid.uuid4()),
                    "text": text,
                    "deliberation": deliberacao.lower(),
                    "services": ", ".join(service_info["servicos"]).lower(),
                    "location": llm_output.get("localidade", "")
                })

# Convert the list of dictionaries to a pandas dataframe
df = pd.DataFrame(data)

# Save the dataframe to a CSV file
df.to_csv('data/training_data.csv', index=False, encoding='utf-8')
df.shape


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

(106, 5)

In [23]:
data = pd.read_csv('data/training_data.csv')
data.iloc[6]





id                           62327bf5-9972-4158-998e-c716fcb460e0
text             ITAQUAQ INVESTIMENTOS IMOBILIÁRIOS E PARTICIP...
deliberation                          alteração do capital social
services        coordenação do registro de ato societário pera...
location                                             São Paulo/SP
Name: 6, dtype: object

In [3]:
from transformers import T5Tokenizer

# Carregar o tokenizer do modelo ptt5-base-portuguese-vocab
tokenizer = T5Tokenizer.from_pretrained('unicamp-dl/ptt5-base-portuguese-vocab', legacy=False)

# Criar a função de comprimento de tokens
def t5_token_len(text):
    tokens = tokenizer.encode(
        text,
        add_special_tokens=False  # Remove tokens especiais que podem não ser necessários
    )
    return len(tokens)

# Testar a função
text = "hello I am a chunk of text and using the t5_token_len function we can find the length of this chunk of text in tokens"
print(t5_token_len(text))


46


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=t5_token_len,
    separators=["\n\n", "\n", " ", ""]
)

In [5]:
chunks = text_splitter.split_text(data.iloc[2]['text'])
print(len(chunks))
print(chunks[0])
print(chunks[1])
print(chunks[2])
t5_token_len(chunks[0]), t5_token_len(chunks[1]), t5_token_len(chunks[2])


11
1 CH4 ENERGIA LTDA. CNPJ n 29.199.9330001-31 NIRE 33.2.1229294-1 ATA DA REUNIÃO DE SÓCIOS REALIZADA EM 19 DE MAIO DE 2023 1. DATA, HORA E LOCAL Realizada no dia 19 de maio de 2023, às 11h00, na sede social da CH4 ENERGIA LTDA., localizada na Cidade do Rio de Janeiro, Estado do Rio de Janeiro, na Rua Humaitá, n 275, 8 andar, Humaitá, CEP 22.261-005 (Sociedade). 2. CONVOCAÇÃO E PRESENÇA Dispensadas as formalidades de convocação da reunião, tendo em vista a presença da única sócia da Sociedade, representando a totalidade do capital social da Sociedade, nos termos do Art. 1.072, 2 da Lei n 10.40602 (Código Civil), a saber NFE BRAZIL HOLDINGS LLC, sociedade devidamente constituída e existente de acordo com as leis de Delaware, Estados Unidos da América, com sede em 1209 Orange Street, Wilmington, Delaware 19801, Estados Unidos da América, inscrita no Cadastro Nacional da Pessoa Jurídica (CNPJ) sob o n 40.302.8910001-55, neste ato representada por seu representante legal, o Sr. Jeremy Pau

(499, 500, 497)

In [5]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')


In [7]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embeddings.embed_documents(texts)
len(res), len(res[0])

(2, 768)

In [3]:
pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

index_name = "document-index"

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=768, 
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 953}},
 'total_vector_count': 953}

In [9]:
batch_limit = 100

texts = []
metadatas = []

for i, record in data.iterrows():
    # first get metadata fields for this record
    metadata = {
        'doc-id': record['id'],
        'deliberation': record['deliberation'],
        'services': record['services'],
        'location': record['location'],
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['text'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid.uuid4()) for _ in range(len(texts))]
        embeds = embeddings.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []
print(len(texts))
if len(texts) > 0:
    ids = [str(uuid.uuid4()) for _ in range(len(texts))]
    embeds = embeddings.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

68


In [10]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 953}},
 'total_vector_count': 953}

In [15]:
vectorstore = LangChainPinecone(index, embeddings.embed_query, "text")
llm = HuggingFaceHub(repo_id="unicamp-dl/ptt5-base-portuguese-vocab", task='text2text-generation')


# Configurar RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

/Users/pedrorenan/.pyenv/versions/3.11.4/envs/ironhack/lib/python3.11/site-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [18]:
def ask_question(question):
    return qa.run(question)

In [19]:
question = "Quais são as deliberações e localização para o ato societário?"
answer = ask_question(question)
print(answer)

Use the following pieces of context to answer the ques


In [20]:
# Dados das deliberações
deliberations = [
    {
        "title": "Abertura de empresa",
        "description": "Procedimentos para registrar e iniciar uma empresa."
    },
    {
        "title": "Alteração de Razão Social",
        "description": "Processo de mudança do nome legal de uma empresa."
    },
    {
        "title": "Alteração do endereço",
        "description": "Atualização do endereço comercial de uma empresa nos registros oficiais."
    },
    {
        "title": "Alteração de objeto",
        "description": "Modificação da atividade econômica ou do propósito empresarial registrado."
    },
    {
        "title": "Abertura de Filial",
        "description": "Procedimentos para estabelecer uma nova localização ou extensão da empresa."
    },
    {
        "title": "Alteração de Capital Social",
        "description": "Ajuste no valor do investimento financeiro na estrutura da empresa."
    },
    {
        "title": "Alteração de quadro societário",
        "description": "Tipo de empresa, nome, endereço, atividades comerciais, sócios, entre outros, conforme exigido pela legislação local."
    },
    {
        "title": "Distrato/Encerramento",
        "description": "Processo de dissolução ou fechamento formal de uma empresa."
    }
]

# Dados dos serviços
services = [
    {
        "title": "Viabilidade",
        "description": "Análise e coordenação para verificar a viabilidade de um empreendimento perante a Prefeitura, garantindo conformidade com regulamentações municipais."
    },
    {
        "title": "Emissão de Documento Básico de Entrada (DBE)",
        "description": "Documento necessário para iniciar a abertura de uma empresa junto à Receita Federal. Envolve fornecer informações básicas sobre a empresa e seus sócios."
    },
    {
        "title": "Registro de ato societário perante a Junta Comercial",
        "description": "Processo para formalizar a constituição ou qualquer alteração na estrutura da empresa perante a Junta Comercial. Inclui definição de sócios, capital social e endereço."
    },
    {
        "title": "Inscrição do cadastro municipal",
        "description": "Registro obrigatório para que a empresa possa operar localmente e cumprir suas obrigações fiscais municipais."
    },
    {
        "title": "Atualização do cadastro municipal",
        "description": "Processo de atualização dos dados da empresa junto à prefeitura para garantir conformidade legal e manter registros atualizados."
    },
    {
        "title": "Baixa do cadastro municipal",
        "description": "Encerramento formal do registro da empresa perante a prefeitura, geralmente realizado quando a empresa encerra suas atividades."
    },
    {
        "title": "Inscrição do FGTS da sociedade na Caixa Econômica Federal",
        "description": "Registro para garantir os direitos trabalhistas dos funcionários da empresa. Envolve inscrever a empresa no Fundo de Garantia por Tempo de Serviço (FGTS)."
    },
    {
        "title": "Atualização do FGTS da sociedade na Caixa Econômica Federal",
        "description": "Processo de atualização de informações relacionadas ao Fundo de Garantia por Tempo de Serviço para manter conformidade legal e dados atualizados."
    },
    {
        "title": "Baixa do FGTS da sociedade na Caixa Econômica Federal",
        "description": "Encerramento formal do registro do Fundo de Garantia por Tempo de Serviço da empresa, geralmente realizado quando a empresa encerra suas atividades."
    },
    {
        "title": "Obtenção de inscrição estadual",
        "description": "Registro necessário para que a empresa possa realizar operações comerciais dentro do estado. Envolve inscrever a empresa na Secretaria da Fazenda Estadual."
    },
    {
        "title": "Atualização de inscrição estadual",
        "description": "Processo de atualização das informações do registro estadual da empresa para manter conformidade legal e registros atualizados."
    },
    {
        "title": "Baixa de inscrição estadual",
        "description": "Encerramento formal do registro estadual da empresa, geralmente realizado quando a empresa encerra suas atividades ou não opera mais dentro do estado."
    },
    {
        "title": "Emissão de alvará de funcionamento",
        "description": "Documento que autoriza legalmente a empresa a operar em determinada localidade. Geralmente emitido pela prefeitura local após verificação de conformidade com regulamentações."
    }
]

# Relação entre deliberações e serviços
relations = {
    "Abertura de empresa": ["Viabilidade", "Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial", "Inscrição do cadastro municipal", "Inscrição do FGTS da sociedade na Caixa Econômica Federal", "Obtenção de inscrição estadual", "Emissão de alvará de funcionamento"],
    "Alteração de Razão Social": ["Viabilidade", "Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial", "Atualização do cadastro municipal", "Atualização do FGTS da sociedade na Caixa Econômica Federal", "Emissão de alvará de funcionamento"],
    "Alteração do endereço": ["Viabilidade", "Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial", "Atualização do cadastro municipal", "Atualização do FGTS da sociedade na Caixa Econômica Federal", "Atualização de inscrição estadual", "Emissão de alvará de funcionamento"],
    "Alteração de objeto": ["Viabilidade", "Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial", "Atualização do cadastro municipal", "Atualização de inscrição estadual", "Emissão de alvará de funcionamento"],
    "Abertura de Filial": ["Viabilidade", "Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial", "Inscrição do cadastro municipal", "Inscrição do FGTS da sociedade na Caixa Econômica Federal", "Obtenção de inscrição estadual", "Emissão de alvará de funcionamento"],
    "Alteração de quadro societário": ["Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial", "Atualização do cadastro municipal"],
    "Alteração de Capital Social": ["Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial"],
    "Distrato/Encerramento": ["Emissão de Documento Básico de Entrada (DBE)", "Registro de ato societário perante a Junta Comercial", "Baixa do cadastro municipal", "Baixa do FGTS da sociedade na Caixa Econômica Federal", "Baixa de inscrição estadual"]
}


In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1"
)

index_name = "deliberations-services"
dimension = 768
metric = "cosine"

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]
# Verificar se o índice já existe, caso contrário, criá-lo
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=spec
    )

index = pc.Index(index_name)

In [13]:
# Inicializar o modelo de embeddings
model = SentenceTransformer('neuralmind/bert-base-portuguese-cased')

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


In [25]:
vector_data = []

# Adicionar deliberações com relações como metadados
for delib in deliberations:
    vector_data.append({
        "id": str(uuid.uuid4()),
        "title": delib["title"],
        "description": delib["description"],
        "category": "deliberation",
        "relations": relations.get(delib["title"], [])
    })

# Adicionar serviços
for serv in services:
    vector_data.append({
        "id": str(uuid.uuid4()),
        "title": serv["title"],
        "description": serv["description"],
        "category": "service",
        "relations": []  # Serviços não têm relações adicionais
    })



# Gerar embeddings para os dados
vectors = []
for item in vector_data:
    vector = model.encode(item["description"]).tolist()
    metadata = {
        "title": item["title"],
        "description": item["description"],
        "category": item["category"],
        "relations": item["relations"]
    }
    vectors.append((item["id"], vector, metadata))

# Inserir vetores na index
index.upsert(vectors)

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


{'upserted_count': 21}

In [53]:
from langchain.globals import set_debug
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.agents import Tool, AgentExecutor
from langchain.chains.question_answering import load_qa_chain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType




set_debug(True)


# llm = HuggingFaceHub(repo_id='google/flan-t5-base', task='text2text-generation')



embeddings = HuggingFaceEmbeddings(model_name='neuralmind/bert-base-portuguese-cased')

# Configurar o vetorstore do Pinecone
vectorstore = LangChainPinecone(
    index=index,
    embedding=embeddings,
    text_key="description"
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

deliberations = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)




No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


In [55]:


tools = [
    Tool(
        name = "Deliberations",
        func=deliberations.run,
        description="""
        Use para saber quais são as deliberações que devem ser tomadas para um ato societário específico.
        Deverá retornar apelas a lista das deliberações possíveis que deve corresponder ao campo title do documento encontrado no pinecone.
        """
    )
]

agent_kwargs = {'prefix': f'''
                Você é um especialista em atos societários e pode ajudar a responder perguntas sobre eles.
                Sempre que receber um texto de um ato societário, você deve pesquisar quais são as deliberações possíveis e a cidade e estado onde elas devem ser realizadas.
                '''}

agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True, 
                         agent_kwargs=agent_kwargs,
                         handle_parsing_errors=True
                         )
# define q and a function for frontend

response = agent.run(data.iloc[2]['text'])
print(response)


[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "1 CH4 ENERGIA LTDA. CNPJ n 29.199.9330001-31 NIRE 33.2.1229294-1 ATA DA REUNIÃO DE SÓCIOS REALIZADA EM 19 DE MAIO DE 2023 1. DATA, HORA E LOCAL Realizada no dia 19 de maio de 2023, às 11h00, na sede social da CH4 ENERGIA LTDA., localizada na Cidade do Rio de Janeiro, Estado do Rio de Janeiro, na Rua Humaitá, n 275, 8 andar, Humaitá, CEP 22.261-005 (Sociedade). 2. CONVOCAÇÃO E PRESENÇA Dispensadas as formalidades de convocação da reunião, tendo em vista a presença da única sócia da Sociedade, representando a totalidade do capital social da Sociedade, nos termos do Art. 1.072, 2 da Lei n 10.40602 (Código Civil), a saber NFE BRAZIL HOLDINGS LLC, sociedade devidamente constituída e existente de acordo com as leis de Delaware, Estados Unidos da América, com sede em 1209 Orange Street, Wilmington, Delaware 19801, Estados Unidos da América, inscrita no Cadastro Nacional da Pessoa Jurídica (CNPJ) sob o n 40.302.89